# Action Menu - Bob

## Role - Responder

In this notebook we'll be going through the Action Menu Protocol. For details on the protocol, please refer to the [RFC](https://github.com/hyperledger/aries-rfcs/tree/master/features/0509-action-menu). Specifically, we'll demonstrate how Bob acts as a Responder to respond to Action Menu related requests from Alice.

A requirement for this protocol to work is an active DIDComm communication channel between Alice and Bob. To achieve that, you should have completed the [Establishing a Connection](http://127.0.0.1:8888/notebooks/Part%203%20-%20Establishing%20a%20Connection.ipynb) tutorial.

## 1. Initialise Bob controller

We begin by initialising the controller for Bob.

In [ ]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

## 2. Listen for webhooks and register default listeners

Everytime a webhook is received from the agent, the controller reemits the hook using [PyPubSub](https://pypubsub.readthedocs.io/en/v4.0.3/). The default listeners are used to update state and print logs.

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

In [ ]:
def get_menu_handler(payload):
    print(f"Get Active Menu Webhook: {payload}")

def perform_menu_handler(payload):
    print("Perform Menu Action Webhook: ", payload)

get_menu_listener = {
    "topic": "get-active-menu",
    "handler": get_menu_handler
}

perform_menu_listener = {
    "topic": "perform-menu-action",
    "handler": perform_menu_handler
}

agent_controller.register_listeners([get_menu_listener, perform_menu_listener], defaults=True)

## 3. Check the agent has an active connection

An active connection between Alice and Bob is required. You can either:

- complete the [Establishing a Connection](http://127.0.0.1:8888/notebooks/Part%203%20-%20Establishing%20a%20Connection.ipynb) tutorial, or
- running the python script `create_connection.py` in the setup folder

In [ ]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")

## 6. Send Action Menu to Alice

Bob responds to Alice's request by sending over the action menu that Bob supports.

In [ ]:
# feel free to edit the menu options.
menu_options = [
      {
        "name": "faber_college_actions",
        "title": "Faber College Actions",
        "description": "Faber College Actions",
        "disabled": "False",
        "form": {
          "title": "Actions",
          "description": "Faber College Actions",
          "params": [{
              "name": "remove_cred_exchange_record",
              "title": "Remove Credential Exchange Record",
              "type": "boolean",
              "required": False
          }, {
              "name": "begin_issue_cred",
              "title": "Begin Issue Credential",
              "type": "boolean",
              "required": False
          }],
          "submit-label": "Send"
        }
      }
    ]

await agent_controller.action_menu.send_menu(
    connection_id=connection_id,
    menu_description="Menu for performing actions with Faber College",
    menu_errormsg="Error: something went wrong",
    menu_title="Faber College Action Menu",
    menu_options=menu_options
)

## 7. Continue with step 8 of [Alice's notebook](http://127.0.0.1:8889/notebooks/Part%209%20-%20Action%20Menu.ipynb)

## End of Tutorial

Be sure to terminate the controller before running another tutorial.

In [ ]:
response = await agent_controller.terminate()
print(response)